In [1]:
import torch
import timeit
import matplotlib.pyplot as plt
import numpy as np

from pprint import pprint
from copy import deepcopy as dc

import pandas as pd 
import wandb
api = wandb.Api()

from pathlib import Path
home = str(Path.home())

# Define the standard plotting colours.
color_names = [
    "tab:blue",
    "tab:orange",
    "tab:green",
    "tab:red",
    "tab:purple",
    "tab:brown",
    "tab:pink",
    "tab:gray",
    "tab:olive",
    "tab:cyan"
]

color_names = color_names * 10

In [2]:
# # Project is specified by <entity/project-name>
# runs = api.runs("andrewwarrington/fivo-aux-beta")

# summary_list, config_list, name_list = [], [], []
# for run in runs: 
#     # .summary contains the output keys/values for metrics like accuracy.
#     #  We call ._json_dict to omit large files 
#     summary_list.append(run.summary._json_dict)

#     # .config contains the hyperparameters.
#     #  We remove special values that start with _.
#     config_list.append(
#         {k: v for k,v in run.config.items()
#           if not k.startswith('_')})

#     # .name is the human-readable name of the run.
#     name_list.append(run.name)

# runs_df = pd.DataFrame({
#     "summary": summary_list,
#     "config": config_list,
#     "name": name_list
#     })

# runs_df.to_csv("project.csv")

# Get individual run.

In [3]:
def get_experiment_from_wandb(project, exp_tag, exp_key=None, exp_val=None):

    # Grab all the runs.
    runs = api.runs(project)
    filtered_runs = []
    filtered_histories = []
    for _run in runs: 
        try:
            # if _run.config['group'] == exp_tag:
            if _run.group == exp_tag:
                filtered_runs.append(_run)
                filtered_histories.append(_run.history(samples=99999999))  # Run with some token large number to grab all the samples.  Can be very slow.
        except:
            pass
    print(len(filtered_runs))
    
    # Throw and error if there were no runs.
    if len(filtered_runs) == 0:
        raise RuntimeError("No experiments found.  Check the exp_tag.")


    # Now re-grab the runs and make sure they all share keys.
    key_runs = api.runs(project)
    keys = []
    for _run in key_runs: 
        try:
            if _run.group == exp_tag:
                keys.append(_run.scan_history().__next__().keys())
        except Exception as err:
            pass

    print(keys)
    assert all([keys[0] == _k for _k in keys]), 'Keys are not shared.'
    
    
    # Go and get everything.
    values = []
    lengths = []
    for _hist in filtered_histories:
        _values = []
        for _v in _hist:
            _values.append(_v)
        values.append(_values)
        lengths.append(len(_values))
    min_Length = np.min(lengths)
        
    return values, min_Length, filtered_histories


In [4]:
# filtered_histories = run.history(samples=99999999)  # Run with some token large number to grab all the samples.  Can be very slow.
# filtered_histories = [_r.scan_history() for _r in filtered_runs]  # Default to using fewer samples.

plot_styles = ['-', '--', ':', '-.'] * 10

# Project is specified by <entity/project-name>
project = "andrewwarrington/fivo-aux-beta"

# exp_tag = ['gdm-v2.1.0-sgr-pqr', ]
# save_tag = 'gdm-v2-1-1'

# exp_tag = ['gdm-v2.1.0-sgr-pqr', 'gdm-v2.1.0-nsgr-pqr']
# save_tag = 'gdm-v2-1'

# exp_tag = ['gdm-v2.0.0-sgr', 'gdm-v2.0.0-no-sgr', ]
# save_tag = 'gdm-v2-0

exp_tag = ['debug', ]
save_tag = 'debug'

filtered_histories = []
for _tag in exp_tag:
    _, _, _filtered_histories = get_experiment_from_wandb(project, _tag)
    filtered_histories.append(_filtered_histories)

pprint("Keys:")
pprint(list(filtered_histories[0][0].keys()))

144
[dict_keys(['step', 'small_lml_variance_fivo', '_step', 'params_r_pred.head_mean_fn_kernel', 'params_p_true', 'small_lml_variance_bpf_true', 'params_r_pred.head_mean_fn_bias', 'params_q_pred.head_mean_fn_kernel', 'params_r_pred.head_log_var_fn_bias', 'params_q_pred.head_log_var_fn_bias', 'small_lml_mean_em_true', 'params_p_pred', '_runtime', 'small_fivo_bound', 'epoch', 'small_lml_mean_bpf_true', 'pred_fivo_bound', 'pred_lml', 'small_lml_mean_fivo', '_timestamp']), dict_keys(['step', 'small_lml_variance_fivo', '_step', 'params_r_pred.head_mean_fn_kernel', 'params_p_true', 'small_lml_variance_bpf_true', 'params_r_pred.head_mean_fn_bias', 'params_q_pred.head_mean_fn_kernel', 'params_r_pred.head_log_var_fn_bias', 'params_q_pred.head_log_var_fn_bias', 'small_lml_mean_em_true', 'params_p_pred', '_runtime', 'small_fivo_bound', 'epoch', 'small_lml_mean_bpf_true', 'pred_fivo_bound', 'pred_lml', 'small_lml_mean_fivo', '_timestamp']), dict_keys(['step', 'small_lml_variance_fivo', '_step', 'p

AssertionError: Keys are not shared.

In [ ]:
# all_keys_to_plot = list(filtered_histories[0][0].keys())  # ['params_q_pred.head_mean_fn_kernel',]  # {list(filtered_histories[0][0].keys()), [STR,]}

# for key_to_plot in all_keys_to_plot:

#     x = filtered_histories[0][0].get('step').values.squeeze()
    
#     try:
#         f = plt.figure(figsize=(6,6))

#         for _i, _k in enumerate(exp_tag):

#             to_plot = []
#             for _j in range(len(filtered_histories[_i])):
#                 to_plot.append(np.stack(filtered_histories[_i][_j].get(key_to_plot).values.squeeze()))

#             h = np.stack(to_plot).squeeze()
#             mu = np.quantile(h, 0.50, axis=0)
#             lq = np.quantile(h, 0.25, axis=0)
#             uq = np.quantile(h, 0.75, axis=0)

#             mu = mu.reshape(len(mu), -1)
#             lq = lq.reshape(len(lq), -1)
#             uq = uq.reshape(len(uq), -1)

#             for _j in range(len(mu.T)):
#                 plt.plot(x, mu[:, _j], c=color_names[_j], linestyle=plot_styles[_i])
#                 plt.fill_between(x, lq[:, _j], uq[:, _j], color=color_names[_j], alpha=0.05)
#             plt.grid(True)

#         plt.xlabel(['{}: {}'.format(_a, _b) for _a, _b in zip(exp_tag, plot_styles)], fontsize=17)
#         plt.title(key_to_plot, fontsize=17)

#         plt.savefig('{}/Downloads/{}_{}.pdf'.format(home, save_tag, key_to_plot))
#         plt.pause(0.1)
#     except Exception as err:
#         print(key_to_plot, 'failed.')
#         print(err)
#         plt.close(f)

In [ ]:
plt.plot(x, mu[:, _j], c=color_names[_j], linestyle=plot_styles[_i])

In [ ]:
_i

In [ ]:
filtered_histories[_i][_j].get(key_to_plot).values[1:]

In [ ]:
filtered_histories[_i][_j].get(key_to_plot).values[1:]

In [ ]:
print(_i)

In [ ]:
h.shape

In [ ]:
_i = 0
print(np.arange(10)[_i])

In [5]:
runs = api.runs(project)
[_r.name == 'fragrant-breeze-370' for _r in runs]

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 

In [6]:
p = runs[0].history(samples=99999999)

In [7]:
p

,step,small_lml_variance_fivo,_step,params_r_pred.head_mean_fn_kernel,params_p_true,small_lml_variance_bpf_true,params_r_pred.head_mean_fn_bias,params_q_pred.head_mean_fn_kernel,params_r_pred.head_log_var_fn_bias,params_q_pred.head_log_var_fn_bias,small_lml_mean_em_true,params_p_pred,_runtime,small_fivo_bound,epoch,small_lml_mean_bpf_true,pred_fivo_bound,pred_lml,small_lml_mean_fivo,_timestamp
0,1,6.116892e-02,1,"[[[-0.11227913945913315]], [[-0.01658167690038...",None,0.003489,"[[0.057596754282712936], [-0.06059430539608002...","[[[0.19928792119026184], [0.01687648892402649]...","[[0.9984650015830994], [0.9983424544334412], [...","[[1.0000019073486328], [1.0010806322097778], [...",None,None,47,3.455400,1,2.46767,3.059108,2.507343,2.675033,1639773010
1,250,6.189129e-02,250,"[[[-0.11227913945913315]], [[-0.01658167690038...",None,0.003489,"[[0.057596754282712936], [-0.06059430539608002...","[[[0.19928792119026184], [0.01687648892402649]...","[[1.593528151512146], [1.6259125471115112], [1...","[[1.0000019073486328], [1.0010806322097778], [...",None,None,52,3.619771,250,2.46767,2.957047,2.602430,2.627870,1639773015
2,500,2.197207e-01,500,"[[[-0.11227913945913315]], [[-0.01658167690038...",None,0.003489,"[[0.057596754282712936], [-0.06059430539608002...","[[[0.19928792119026184], [0.01687648892402649]...","[[2.0368988513946533], [2.1239662170410156], [...","[[1.0000019073486328], [1.0010806322097778], [...",None,None,58,3.391365,500,2.46767,2.943288,2.573095,2.567509,1639773021
3,750,8.983465e-02,750,"[[[-0.11227913945913315]], [[-0.01658167690038...",None,0.003489,"[[0.057596754282712936], [-0.06059430539608002...","[[[0.19928792119026184], [0.01687648892402649]...","[[2.4224772453308105], [2.5639488697052], [2.6...","[[1.0000019073486328], [1.0010806322097778], [...",None,None,63,3.066481,750,2.46767,2.787473,2.499227,2.547787,1639773026
4,1000,2.061969e-02,1000,"[[[-0.11227913945913315]], [[-0.01658167690038...",None,0.003489,"[[0.057596754282712936], [-0.06059430539608002...","[[[0.19928792119026184], [0.01687648892402649]...","[[2.7609071731567383], [2.949091911315918], [3...","[[1.0000019073486328], [1.0010806322097778], [...",None,None,68,2.996767,1000,2.46767,2.736491,2.483878,2.459852,1639773031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,99000,9.304076e-07,99000,"[[[-0.11227913945913315]], [[-0.01658167690038...",None,0.003489,"[[0.057596754282712936], [-0.06059430539608002...","[[[0.19928792119026184], [0.01687648892402649]...","[[10.029670715332031], [9.0275297164917], [8.0...","[[1.0000019073486328], [1.0010806322097778], [...",None,None,3793,2.672155,99000,2.46767,2.670807,2.458439,2.458568,1639776756
397,99250,7.016198e-07,99250,"[[[-0.11227913945913315]], [[-0.01658167690038...",None,0.003489,"[[0.057596754282712936], [-0.06059430539608002...","[[[0.19928792119026184], [0.01687648892402649]...","[[10.005931854248047], [9.001008033752441], [8...","[[1.0000019073486328], [1.0010806322097778], [...",None,None,3799,2.668360,99250,2.46767,2.670759,2.458282,2.458208,1639776762
398,99500,5.782629e-07,99500,"[[[-0.11227913945913315]], [[-0.01658167690038...",None,0.003489,"[[0.057596754282712936], [-0.06059430539608002...","[[[0.19928792119026184], [0.01687648892402649]...","[[10.065546035766602], [9.056654930114746], [8...","[[1.0000019073486328], [1.0010806322097778], [...",None,None,3804,2.673264,99500,2.46767,2.670783,2.458433,2.458377,1639776767
399,99750,6.577137e-07,99750,"[[[-0.11227913945913315]], [[-0.01658167690038...",None,0.003489,"[[0.057596754282712936], [-0.06059430539608002...","[[[0.19928792119026184], [0.01687648892402649]...","[[10.05737018585205], [9.047799110412598], [8....","[[1.0000019073486328], [1.0010806322097778], [...",None,None,3810,2.672183,99750,2.46767,2.671037,2.458552,2.458349,1639776773


In [10]:
np.stack(p.get('params_r_pred.head_log_var_fn_bias').values).squeeze()[-200:].mean(axis=0)

array([10.04317357,  9.04084138,  8.04120706,  7.04101599,  6.03995616,
        5.03942546,  4.04014838,  3.03478388,  2.02281377])